In [2]:
import pandas as pd
import os
import gc
import time
import csv
import sqlite3

### NPI files

In [16]:
data_dir = "nppes_zip_files_v2"

In [14]:
pd.read_csv("nppes_zip_files_v2/NPPES_Data_Dissemination_050525_051125_Weekly_V2/npidata_pfile_20250505-20250511_fileheader.csv",
           header = None).values[0].tolist()  

['NPI',
 'Entity Type Code',
 'Replacement NPI',
 'Employer Identification Number (EIN)',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider Other Organization Name',
 'Provider Other Organization Name Type Code',
 'Provider Other Last Name',
 'Provider Other First Name',
 'Provider Other Middle Name',
 'Provider Other Name Prefix Text',
 'Provider Other Name Suffix Text',
 'Provider Other Credential Text',
 'Provider Other Last Name Type Code',
 'Provider First Line Business Mailing Address',
 'Provider Second Line Business Mailing Address',
 'Provider Business Mailing Address City Name',
 'Provider Business Mailing Address State Name',
 'Provider Business Mailing Address Postal Code',
 'Provider Business Mailing Address Country Code (If outside U.S.)',
 'Provider Business Mailing Address Telephone Nu

#### Step 1: Gather all the files together and delete all the useless information

In [17]:
ls_file = []
ls_dir = []
dic_fileheader = {}
for dir_name in os.listdir(data_dir):
    path = os.path.join(data_dir, dir_name)
    if os.path.isdir(path):
        for filename in os.listdir(path):

            # delete all "fileheader", they contain no information
            if "fileheader" in filename:
                file_path = os.path.join(data_dir,dir_name, filename)
                dic_fileheader[filename] = pd.read_csv(file_path, header=None).values[0].tolist()
                try:
                    os.remove(file_path)
                    print(f"Deleted: {filename}")
                except Exception as e:
                    print(f"Failed to delete {filename}: {e}")

            else:
                # delete duplicate files
                if filename in ls_file:
                    file_path = os.path.join(data_dir,dir_name, filename)
                    try:
                        os.remove(file_path)
                        print(f"Deleted: {filename}")
                    except Exception as e:
                        print(f"Failed to delete {filename}: {e}")
                # move other files to the main directory
                else:
                    ls_file.append(filename)
                    ls_dir.append(dir_name)
                    source_path = os.path.join(data_dir, dir_name, filename)
                    destination_path = os.path.join(data_dir, filename)        
                    os.rename(source_path, destination_path)

        # Remove the empty directory
        os.rmdir(os.path.join(data_dir, dir_name))  
            
        print(f"Processing directory: {dir_name}")

Deleted: endpoint_pfile_20250505-20250511_fileheader.csv
Deleted: npidata_pfile_20250505-20250511_fileheader.csv
Deleted: othername_pfile_20250505-20250511_fileheader.csv
Deleted: pl_pfile_20250505-20250511_fileheader.csv
Processing directory: NPPES_Data_Dissemination_050525_051125_Weekly_V2
Deleted: endpoint_pfile_20250512-20250518_fileheader.csv
Deleted: npidata_pfile_20250512-20250518_fileheader.csv
Deleted: NPPES_Data_Dissemination_CodeValues.pdf
Deleted: NPPES_Data_Dissemination_Readme_v.2.pdf
Deleted: othername_pfile_20250512-20250518_fileheader.csv
Deleted: pl_pfile_20250512-20250518_fileheader.csv
Processing directory: NPPES_Data_Dissemination_051225_051825_Weekly_V2
Deleted: endpoint_pfile_20250519-20250525_fileheader.csv
Deleted: npidata_pfile_20250519-20250525_fileheader.csv
Deleted: NPPES_Data_Dissemination_CodeValues.pdf
Deleted: NPPES_Data_Dissemination_Readme_v.2.pdf
Deleted: othername_pfile_20250519-20250525_fileheader.csv
Deleted: pl_pfile_20250519-20250525_fileheader.

In [24]:
df_files = pd.DataFrame({"filename": ls_file, "dir": ls_dir})

dic_fileheader_cleaned = {
    key.replace("_fileheader", ""): value
    for key, value in dic_fileheader.items()
}
df_files['fileheader'] = df_files['filename'].map(dic_fileheader_cleaned)

df_files.to_csv("nppes_zip_files_v2.csv", index=False)

#### Step 2: data integration

In [36]:
df_files = pd.read_csv("nppes_zip_files_v2.csv")
conn = sqlite3.connect("nppes.db")
batch_size = 50000

In [37]:
table_name = "nppes_endpoint"
for filename in df_files.query("filename.str.startswith('endpoint')")["filename"]:
    input_file = os.path.join(data_dir, filename)

    print(f"Processing file: {filename}")

    with open(input_file, mode="r", encoding="utf-8", newline='') as f:
        reader = csv.reader(f)
        header = next(reader)  # get header

        batch = []
        batch_num = 1
        for i, row in enumerate(reader, 1):
            batch.append(row)

            if i % batch_size == 0:
                df = pd.DataFrame(batch, columns=header)
                df.to_sql(table_name, conn, if_exists='append', index=False)

                batch_num += 1
                batch.clear()

                gc.collect()
                time.sleep(1)  # Optional: sleep to avoid overwhelming the system

        if batch:
            df = pd.DataFrame(batch, columns=header)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            gc.collect()
            print(f"Saved final batch {batch_num} with {len(batch)} rows for file {input_file}")
    
    time.sleep(1)  # Optional: sleep to avoid overwhelming the system
    os.remove(input_file)
    gc.collect()
    del df
    print(f"Finished processing file: {filename}")
gc.collect()

Processing file: endpoint_pfile_20250505-20250511.csv
Saved final batch 1 with 2182 rows for file nppes_zip_files_v2\endpoint_pfile_20250505-20250511.csv
Finished processing file: endpoint_pfile_20250505-20250511.csv
Processing file: endpoint_pfile_20250512-20250518.csv
Saved final batch 1 with 1981 rows for file nppes_zip_files_v2\endpoint_pfile_20250512-20250518.csv
Finished processing file: endpoint_pfile_20250512-20250518.csv
Processing file: endpoint_pfile_20250519-20250525.csv
Saved final batch 1 with 1937 rows for file nppes_zip_files_v2\endpoint_pfile_20250519-20250525.csv
Finished processing file: endpoint_pfile_20250519-20250525.csv
Processing file: endpoint_pfile_20050523-20250511.csv
Saved final batch 12 with 22803 rows for file nppes_zip_files_v2\endpoint_pfile_20050523-20250511.csv
Finished processing file: endpoint_pfile_20050523-20250511.csv


0

In [48]:
df_endpoint = pd.read_sql("SELECT DISTINCT * FROM nppes_endpoint", conn)
df_endpoint.to_sql("nppes_endpoint", conn, if_exists='replace', index=False)
del df_endpoint
gc.collect()

518

In [38]:
table_name = "nppes_othername"
for filename in df_files.query("filename.str.startswith('othername')")["filename"]:
    input_file = os.path.join(data_dir, filename)

    print(f"Processing file: {filename}")

    with open(input_file, mode="r", encoding="utf-8", newline='') as f:
        reader = csv.reader(f)
        header = next(reader)  # get header

        batch = []
        batch_num = 1
        for i, row in enumerate(reader, 1):
            batch.append(row)

            if i % batch_size == 0:
                df = pd.DataFrame(batch, columns=header)
                df.to_sql(table_name, conn, if_exists='append', index=False)

                batch_num += 1
                batch.clear()

                gc.collect()
                time.sleep(1)  # Optional: sleep to avoid overwhelming the system

        if batch:
            df = pd.DataFrame(batch, columns=header)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            gc.collect()
            print(f"Saved final batch {batch_num} with {len(batch)} rows for file {input_file}")
    
    time.sleep(1)  # Optional: sleep to avoid overwhelming the system
    os.remove(input_file)
    gc.collect()
    del df
    print(f"Finished processing file: {filename}")

Processing file: othername_pfile_20250505-20250511.csv
Saved final batch 1 with 2440 rows for file nppes_zip_files_v2\othername_pfile_20250505-20250511.csv
Finished processing file: othername_pfile_20250505-20250511.csv
Processing file: othername_pfile_20250512-20250518.csv
Saved final batch 1 with 2972 rows for file nppes_zip_files_v2\othername_pfile_20250512-20250518.csv
Finished processing file: othername_pfile_20250512-20250518.csv
Processing file: othername_pfile_20250519-20250525.csv
Saved final batch 1 with 2551 rows for file nppes_zip_files_v2\othername_pfile_20250519-20250525.csv
Finished processing file: othername_pfile_20250519-20250525.csv
Processing file: othername_pfile_20050523-20250511.csv
Saved final batch 14 with 35146 rows for file nppes_zip_files_v2\othername_pfile_20050523-20250511.csv
Finished processing file: othername_pfile_20050523-20250511.csv


In [49]:
df_othername = pd.read_sql("SELECT DISTINCT * FROM nppes_othername", conn)
df_othername.to_sql("nppes_othername", conn, if_exists='replace', index=False)
del df_othername
gc.collect()

0

In [40]:
table_name = "nppes_pl"
for filename in df_files.query("filename.str.startswith('pl')")["filename"]:
    input_file = os.path.join(data_dir, filename)

    print(f"Processing file: {filename}")

    with open(input_file, mode="r", encoding="utf-8", newline='') as f:
        reader = csv.reader(f)
        header = next(reader)  # get header

        batch = []
        batch_num = 1
        for i, row in enumerate(reader, 1):
            batch.append(row)

            if i % batch_size == 0:
                df = pd.DataFrame(batch, columns=header)
                df.to_sql(table_name, conn, if_exists='append', index=False)

                batch_num += 1
                batch.clear()

                gc.collect()
                time.sleep(1)  # Optional: sleep to avoid overwhelming the system

        if batch:
            df = pd.DataFrame(batch, columns=header)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            gc.collect()
            print(f"Saved final batch {batch_num} with {len(batch)} rows for file {input_file}")
    
    time.sleep(1)  # Optional: sleep to avoid overwhelming the system
    os.remove(input_file)
    gc.collect()
    del df
    print(f"Finished processing file: {filename}")

Processing file: pl_pfile_20250505-20250511.csv
Saved final batch 1 with 8837 rows for file nppes_zip_files_v2\pl_pfile_20250505-20250511.csv
Finished processing file: pl_pfile_20250505-20250511.csv
Processing file: pl_pfile_20250512-20250518.csv
Saved final batch 1 with 9696 rows for file nppes_zip_files_v2\pl_pfile_20250512-20250518.csv
Finished processing file: pl_pfile_20250512-20250518.csv
Processing file: pl_pfile_20250519-20250525.csv
Saved final batch 1 with 8825 rows for file nppes_zip_files_v2\pl_pfile_20250519-20250525.csv
Finished processing file: pl_pfile_20250519-20250525.csv
Processing file: pl_pfile_20050523-20250511.csv
Saved final batch 21 with 18363 rows for file nppes_zip_files_v2\pl_pfile_20050523-20250511.csv
Finished processing file: pl_pfile_20050523-20250511.csv


In [52]:
df_pl = pd.read_sql("SELECT DISTINCT * FROM nppes_pl", conn)
df_pl.to_sql("nppes_pl", conn, if_exists='replace', index=False)
del df_pl
gc.collect()

436

In [ ]:
table_name = "nppes_npi"
for filename in df_files.query("filename.str.startswith('npi')")["filename"]:
    input_file = os.path.join(data_dir, filename)

    print(f"Processing file: {filename}")

    with open(input_file, mode="r", encoding="utf-8", newline='') as f:
        reader = csv.reader(f)
        header = next(reader)  # get header

        batch = []
        batch_num = 1
        for i, row in enumerate(reader, 1):
            batch.append(row)

            if i % batch_size == 0:
                df = pd.DataFrame(batch, columns=header)
                print(f"Saving batch {batch_num} with {len(batch)} rows for file {input_file}")
                df.to_sql(table_name, conn, if_exists='append', index=False)

                batch_num += 1
                batch.clear()

                gc.collect()
                time.sleep(1)  # Optional: sleep to avoid overwhelming the system

        if batch:
            df = pd.DataFrame(batch, columns=header)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            gc.collect()
            print(f"Saved final batch {batch_num} with {len(batch)} rows for file {input_file}")
    
    time.sleep(1)  # Optional: sleep to avoid overwhelming the system
    os.remove(input_file)
    gc.collect()
    del df
    print(f"Finished processing file: {filename}")

Processing file: npidata_pfile_20250505-20250511.csv
Saved final batch 1 with 29166 rows for file nppes_zip_files_v2\npidata_pfile_20250505-20250511.csv
Finished processing file: npidata_pfile_20250505-20250511.csv
Processing file: npidata_pfile_20250512-20250518.csv
Saved final batch 1 with 31320 rows for file nppes_zip_files_v2\npidata_pfile_20250512-20250518.csv
Finished processing file: npidata_pfile_20250512-20250518.csv
Processing file: npidata_pfile_20250519-20250525.csv
Saved final batch 1 with 30333 rows for file nppes_zip_files_v2\npidata_pfile_20250519-20250525.csv
Finished processing file: npidata_pfile_20250519-20250525.csv
Processing file: npidata_pfile_20050523-20250511.csv
Saving batch 1 with 50000 rows for file nppes_zip_files_v2\npidata_pfile_20050523-20250511.csv
Saving batch 2 with 50000 rows for file nppes_zip_files_v2\npidata_pfile_20050523-20250511.csv
Saving batch 3 with 50000 rows for file nppes_zip_files_v2\npidata_pfile_20050523-20250511.csv
Saving batch 4 wi

In [56]:
conn.close()